In [1]:
import sys
import numpy as np
from pathlib import Path
import torchopt
import posteriors

# Add paths for importing utilities and models
current_dir = Path.cwd()
sys.path.append(str(current_dir))
sys.path.append(str(current_dir.parent))
sys.path.append(str(current_dir.parent / "baselines"))


from src.nanogpt_utils import load_model, load_tokenizer, encode, decode
from src.bayesian_utils import create_training_batches, run_bayesian_pipeline
from config import CONFIG, MODEL_PATH, META_PATH, DATA_DIR

[2025-10-05 23:06:47,744] torch.distributed.elastic.multiprocessing.redirects: [WARNING] NOTE: Redirects are currently not supported in Windows or MacOs.
/Users/sofianikolenko/Downloads/Projects_25/ADL/adl-bnn-textgen/.venv/lib/python3.11/site-packages/posteriors/utils.py:9: FutureWarning: The 'optree.integration' module is deprecated and will be removed in version 0.18.0. Please use 'optree.integrations' instead.
  from optree.integration.torch import tree_ravel


Model arguments: {'n_layer': 6, 'n_head': 6, 'n_embd': 384, 'block_size': 256, 'bias': False, 'vocab_size': 65, 'dropout': 0.2}
number of parameters: 10.65M
Model loaded successfully!
Number of parameters: 10,745,088


#### Load pre-trained baseline model

In [2]:
model, checkpoint = load_model(Path(MODEL_PATH))
    
# Load tokenizer and vocabulary size
stoi, itos = load_tokenizer(Path(META_PATH))
vocab_size = len(itos)

# Extract model parameters for posteriors
params = dict(model.named_parameters())

Model arguments: {'n_layer': 6, 'n_head': 6, 'n_embd': 384, 'block_size': 256, 'bias': False, 'vocab_size': 65, 'dropout': 0.2}
number of parameters: 10.65M
Model loaded successfully!
Number of parameters: 10,745,088


#### Preparing training data

In [3]:
# Prepare training data for Bayesian inference
train_data_path = Path(DATA_DIR / 'train.bin')
data = np.memmap(str(train_data_path), dtype=np.uint16, mode='r')

training_batches = create_training_batches(
    data, 
    CONFIG['batch_size'], 
    CONFIG['max_seq_length'], 
    CONFIG['train_samples']
)

print(f"Created {len(training_batches)} training batches")
print(f"Batch shape: {training_batches[0][0].shape}")
print(f"Target shape: {training_batches[0][1].shape}")

# Calculate number of data points for posteriors
num_data = CONFIG['train_samples']
print(f"Total training samples: {num_data}")
    

Created 32 training batches
Batch shape: torch.Size([16, 128])
Target shape: torch.Size([16, 1])
Total training samples: 500


/Users/sofianikolenko/Downloads/Projects_25/ADL/adl-bnn-textgen/src/bayesian_utils.py:37: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:278.)
  x_tensor = torch.tensor(x_batch, dtype=torch.long, device=DEVICE)


#### Setup Variational Inference

In [4]:
state_vi, metrics_vi, eval_vi = run_bayesian_pipeline(training_batches, 'vi')

wandb: Currently logged in as: ssophi-nikol (ssophi-nikol-tum-ai) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: WARNING Using a boolean value for 'reinit' is deprecated. Use 'return_previous' or 'finish_previous' instead.


✓ W&B initialized: https://wandb.ai/ssophi-nikol-tum-ai/bayesian-nanogpt/runs/77srvth6

Setting up VI sampler
✓ VI configured with:
  - Learning rate: 5e-06
  - Temperature: 0.001
  - Samples per update: 1

Starting Bayesian Training with VI
Configuration:
  - Epochs: 3
  - Batches per epoch: 32
  - Total iterations: 96

Epoch 1/3
------------------------------------------------------------
NLL computed successfully: 62.042274475097656
Log prior computed successfully: -15255827.0
Log posterior computed successfully: -30573.697265625
NLL computed successfully: 0.9336455464363098
Log prior computed successfully: -9882597.0
Log posterior computed successfully: -19766.126953125
NLL computed successfully: 58.5260009765625
Log prior computed successfully: -15254245.0
Log posterior computed successfully: -30567.015625
NLL computed successfully: 1.0173876285552979
Log prior computed successfully: -9882596.0
Log posterior computed successfully: -19766.208984375
NLL computed successfully: 65.170

wandb-core(34848) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


NLL computed successfully: 1.0370897054672241
Log prior computed successfully: -9882597.0
Log posterior computed successfully: -19766.23046875
NLL computed successfully: 72.14247131347656
Log prior computed successfully: -15254542.0
Log posterior computed successfully: -30581.2265625


wandb-core(34884) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


...

NLL computed successfully: 0.9465652108192444
Log prior computed successfully: -9882596.0
Log posterior computed successfully: -19766.138671875
  [ 30/32] Loss: 1.2504 | Log Post: -19766.4461
NLL computed successfully: 62.178382873535156
Log prior computed successfully: -15251051.0
Log posterior computed successfully: -30564.279296875
NLL computed successfully: 0.9634807705879211
Log prior computed successfully: -9882597.0
Log posterior computed successfully: -19766.15625
NLL computed successfully: 62.208953857421875
Log prior computed successfully: -15256130.0
Log posterior computed successfully: -30574.46875
NLL computed successfully: 0.5427213907241821
Log prior computed successfully: -9882597.0
Log posterior computed successfully: -19765.736328125
  ✓ Epoch Complete:
    Loss: 1.2237 | Log Post: -19766.4084 | Time: 237.77s

Training Complete!
  Final Loss: 1.2237
  Final Log Posterior: -19766.4084
  Total Time: 667.89s


Evaluating VI Predictions

0. Deterministic Model (Original):

Python(36044) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
wandb: ERROR The nbformat package was not found. It is required to save notebook history.


✓ Model logged to W&B as artifact



batch_log_posterior,▆▆▃▄▃▄▅▆▇▇▆▆▆▃▇▄▃▅▇▄▆▇▅▃▆▁▃▃▅▆▆▆▆▆▇▆▆▆▃█
batch_loss,▃▂█▅▆▃▃▃▄▂▄▃▃▂▅▂▁▇▅▆▄▆▃▄▃▂▂▃▂▆▆▃▅▅▂▁▂▂▂▃
epoch,▁▁▁▁▁▁▁▁▁▁▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅█████████████
epoch_epoch,▁▅█
epoch_log_posterior,█▆▁
epoch_loss,█▁▃
epoch_time,█▁▇
eval/avg_predictive_entropy,▁
eval/deterministic_loss,▁
eval/deterministic_perplexity,▁
+6,...


✓ W&B run finished


#### Run Bayesian Training

#### 8. Compare Deterministic vs Bayesian Predictions